In [1]:
from collections import namedtuple   # assignment   of namedtuple to collections    object
import os


In [2]:
os.getcwd()

'd:\\Image Classification\\Image Classification\\research'

In [3]:
os.chdir("../")

In [4]:
#os.chdir('D:\CNN_claasaifier')

In [5]:
os.getcwd()

'd:\\Image Classification\\Image Classification'

In [6]:
# Creating configuration to craete the directory for data ingestion
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [7]:
from dataclasses import dataclass # specifying data type for variables in python.
from pathlib import Path # to use the system path for   loading data


@dataclass(frozen=True)
class DataIngestionConfig:  # class for creating dataclasses for DataIngestionConfig objects and to craete a directory structure
    root_dir: Path          # path for the directory root
    source_URL: str         # Url for the source file_extension
    local_data_file: Path   # folder for the local data file
    unzip_dir: Path         # folder for unzipping the local data file

In [8]:
!pip install python-box==6.0.2


In [9]:
!pip install joblib

In [10]:
CONFIG_FILE_PATH = Path("configs/config.yaml")   # Creating windows path for config file path for config file_extension
CONFIG_FILE_PATH

WindowsPath('configs/config.yaml')

In [11]:
from deepClassifier.constants import *                            # importing constants
from deepClassifier.utils import read_yaml, create_directories    # importing utils to read the yaml file_extension and create_directories


In [ ]:
from deepClassifier.constants import *

In [ ]:
class ConfigurationManager:                                          # ConfigurationManager classes that inherit from ConfigurationManager
    def __init__(
        self,                                                        # Initialize a variable namedtuple with the following attributes
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:      # crates the data_ingestion_config objects into config files
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:                                                   
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):                                       # download_file returns a zip file containing the data
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):           # it will allow only the .jpg extention with the name "Cat" or "Dog"
        return [f for f in list_of_files if f.endswith(".jpg") and ("Sunny Bhaveen Chandra" in f or "Sudhanshu kumar" in f or "Krish Naik" in f)]  

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e